In [36]:
from bs4 import BeautifulSoup as bs
from requests import get
import requests
import re


Journal of Neurophysiology is structured such that you can find all volume and issue information hidden on any of the volume pages, so we can just start with the first volume and create a list of links for the remaining volumes.

In [ ]:
start_url = 'https://journals.physiology.org/loi/jn/group/d1940.y1940'
url = start_url
year = get(url)
year_page = bs(year.text, 'html.parser')
all_vol = year_page.find_all('a',{'href': re.compile(r'/toc/jn/')})
vol_list = []
for v in all_vol:
    try: 
        vol_str = re.search(r'Volume\s\d{1,3}',str(v)).group()
        vol_int = int(vol_str[7:])
        vol_list.append(vol_int)
    except AttributeError:
        print('Something bad happened.')

vol_list.sort()


Default digital open access starts in Volume 77, issue 1. Perhaps start the loop from this point instead of going through all of the earlier volumes, which are all paywalled and in PDF form.

In [ ]:
open_access = [i for i in vol_list if i>76]
print(open_access)

In [ ]:
last_v = 0
base_url = 'https://journals.physiology.org/toc/jn/'
for v in open_access:
    # count the issues in each volume
    if v!=last_v:
        issue=1
    else:
        issue+=1
    
    # construct the url for the issue and load it
    issue_url = base_url + str(v) +'/' + str(issue)
    print(issue_url)
    get_issue = get(issue_url)
    issue_page = bs(get_issue.text, 'html.parser')
    issue_date = issue_page.find('div',{'class':'col-sm-4 gray-bg toc-right-side'}).\
        find('span',{'class':'coverDate'}).get_text()

    # scrape all of the articles in this issue
    toc = issue_page.find_all('div',{'class':'table-of-content'})[0].find_all('div',{'class':'issue-item'})

    # loop through the articles
    for c in toc:
        section = dict()
        
        # check if article is behind a paywall. If so, skip it (for now)
        if not c.find('div',{'class':'badges'}).get_text():
            continue
        
        # get article metadata
        art.url = 'https://journals.physiology.org' + c.find('a').get('href')
        art.title = c.find('h4').text
        auth = c.find('ul',{'class':'rlist--inline loa'}).find_all('li')
        art.authors = [a.get_text().replace(', and ','') for a in auth]
        art.doi = c.select(".epub-section__item")[0].find('a').get('href')[16:]
        art.id = get_next_id(db_name, table_name) # still needs to be written
        
        # check if this article is already in database
        # (currently, check the title for a match)
        # if not, add metadata to table and load article. If so, continue loop.
        if new_article_check(art.title): # still needs to be written
            add_new_row(db_name, table_name, art)
        else:
            continue
        
        # load article
        get_art = get(art.url)
        art_page = bs(get_art, 'html.parser')
        
        # get reference list
        rlist = art_page.find('ul',{'class':'rlist separator'}).find_all('li')
        # loop through references and add them to database
        for r in rlist:
            r.url = [i.get('href') for i in r.find_all('a')]
            r.title = r.find('span',{'class':'references__article-title'}).get_text()
            r.authors = r.find('span',{'class':'references__authors'}).get_text().split(', ')
            r.id = get_next_id(db_name, table_name) # still needs to be written
            
            # check if article is in database
            if new_article_check(art.title): # still needs to be written
                add_new_row(db_name, table_name, r)
            else:
                continue
            
            # add the citation
            # This function should add the citing ID to a list of citing IDs in the entry for the cited ID
            # It should also add the cited ID to a list of cited IDs in the entry for the citing ID
            add_citation_to_DB(citing_ID = art.id, cited_ID = r.id)
            
            
        # get article sections
        # get abstract
        section{'Abstract'} = art_page.select('div.hlFld-Abstract div.abstractSection')[0].get_text()
        # get all other sections
        section['Introduction'] = ''
        fulltext = test_page.find('div', {'class': 'hlFld-Fulltext'}).findChildren(recursive=False)
        for f in fulltext:
            heading = f.find('h1',{'class':'article-section__title section__title'})
            if not heading:
                section['Introduction']  = section['Introduction']  + ' ' + f.find_text()
            else:
                section[heading.get_text()] = ''
                for text in heading.find_parent().findChildren('div'):
                    section[heading.get_text()] = section[heading.get_text()] + text.get_text()
        
        
            
        
    
    

## Brainstorming:
* Have a function that cycles through the DB, checks for entries with no text, and tries to find the text via PubMed (with the PubMed parser) or with Google Scholar
* 

# Testing stuff out

In [37]:
# page = get('https://journals.physiology.org/doi/full/10.1152/jn.00104.2016')
page = get('https://journals.physiology.org/doi/full/10.1152/jn.00399.2013')
test_page = bs(page.text, 'html.parser')


In [ ]:
loa = test_page.find('div',{'class':'accordion-tabbed loa-accordion'}).\
find_all('div',{'class':'accordion-tabbed__tab-mobile '})
[i.find('a').get_text() for i in loa]

In [ ]:
test_page.select('div.cover-image__details ')[0].find('span',{'class':'volume'}).get_text()

In [ ]:
test_page.select('div.cover-image__details ')[0].find('span',{'class':'issue'}).get_text()

In [ ]:
int(test_page.select('div.cover-image__details ')[0].\
            find('span', {'class': 'coverDate'}).get_text()[-4:])

In [ ]:
fulltext = test_page.find('div', {'class': 'hlFld-Fulltext'}).findChildren(recursive=False)

In [ ]:
fulltext[7].find('h1',{'class':'article-section__title section__title'})

In [ ]:
refs = dict()

In [38]:
rlist = test_page.find('ul', {'class': 'rlist separator'}).find_all('li')

In [39]:
r = rlist[0]
r.find('span', {'class': 'references__year'}).get_text()

'2008'

In [40]:
r.find('span', {'class': 'references__source'}).get_text()

'J Neurophysiol'

In [ ]:
import numpy as np

In [ ]:
np.nan

In [ ]:
section = dict()

In [ ]:
section[fulltext[7].find('h1',{'class':'article-section__title section__title'}).get_text()] = 'hi'

In [ ]:
sec_text = fulltext[7].find('h1',{'class':'article-section__title section__title'}).find_parent().findChildren('div')

In [ ]:
heading = fulltext[7].find('h1',{'class':'article-section__title section__title'})
section[heading.get_text()] = ''
for text in heading.find_parent().findChildren('div'):
    section[heading.get_text()] = section[heading.get_text()] + text.get_text()

## Testing out pymysql with Amazon RDS

In [ ]:
import journal_scrape as js

In [ ]:
# import sys
# !{sys.executable} -m pip install -U PyMySQL
import pymysql

In [ ]:
db = js.Database()

In [ ]:
db.connect()

In [ ]:


# dbname = "findingssm"
# host = "findingssm.c9zjgwsivgee.us-east-2.rds.amazonaws.com"
# port = 3306
# user = "danielkentwood"
# password = "findingsdbsm"

# conn = pymysql.connect(host, user=user, port=port, passwd=password, db=dbname, connect_timeout=5)

In [ ]:
db.cursor()

In [ ]:
sql = '''
    CREATE TABLE [IF NOT EXISTS] metadata (
    id INT,
    url VARCHAR(255),
    year YEAR,
    volume INT,
    issue INT,
    title VARCHAR(255),
    authors VARCHAR(255),
    doi VARCHAR(255)
);'''

db.cursor(sql)

In [ ]:
sql = '''
    CREATE TABLE [IF NOT EXISTS] text (
    id INT,
    sectionA VARCHAR(255),
    sectionB VARCHAR(255),
    text VARCHAR(255)
);'''

db.cursor(sql)

In [57]:
from dbclass import Database
import pandas as pd   

In [58]:
db = Database()
db.create_tables()

In [3]:
 
sql_command = "SELECT * FROM metadata"
# Load the data
data = pd.read_sql(sql_command, db.conn)
data

,id,url,journal,year,volume,issue,title,authors,doi,citations,cited_by


### add article metadata

In [4]:
meta ={}
meta['id'] = 1
meta['url'] = ['article.url']
meta['journal'] = 'self.journal'
meta['year'] = 2001
meta['volume'] = 5
meta['issue'] = 8
meta['title'] = 'Hello world'
meta['authors'] = ['Smith, J','Woodruff, W','Harris, M']
meta['doi'] = 'asdfljasd;flkjasf'
meta['citations'] = [5, 9, 20, 3, 4]
meta['cited_by'] = [1, 2, 6, 7, 8]

sql_command = """\
INSERT INTO metadata (\
id,url,journal,year,volume,issue,title,authors,doi,citations,cited_by\
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

args = (meta['id'], meta['url'],meta['journal'],meta['year'],meta['volume'],
          meta['issue'],meta['title'],meta['authors'],meta['doi'],
          meta['citations'],meta['cited_by'])

with db.conn.cursor() as cursor:
    cursor.execute(sql_command, args)

In [56]:
sql_command = "SELECT * FROM metadata"
# Load the data
data = pd.read_sql(sql_command, db.conn)
data

DatabaseError: Execution failed on sql: SELECT * FROM body
terminating connection due to idle-in-transaction timeout
SSL connection has been closed unexpectedly

unable to rollback

### Update citations

In [6]:
sql = """\
UPDATE metadata \
SET citations = array_cat(citations, %s) \
WHERE id = %s"""
tst = [40, 41, 45]
idnum = 1
args = (tst,idnum)

In [7]:
print(sql)
print(args)

UPDATE metadata SET citations = array_cat(citations, %s) WHERE id = %s
([40, 41, 45], 1)


### add more articles

In [19]:
meta ={}
meta['id'] = 8
meta['url'] = ['article.url']
meta['journal'] = 'self.journal'
meta['year'] = 2008
meta['volume'] = 5
meta['issue'] = 8
meta['title'] = 'Hello world'
meta['authors'] = ['Smith, J','Woodruff, W','Harris, M']
meta['doi'] = 'asdfljasd;flkjasf'
meta['citations'] = [5, 9, 20, 3, 4]
meta['cited_by'] = [1, 2, 6, 7, 8]

sql_command = """\
INSERT INTO metadata (\
id,url,journal,year,volume,issue,title,authors,doi,citations,cited_by\
) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

args = (meta['id'], meta['url'],meta['journal'],meta['year'],meta['volume'],
          meta['issue'],meta['title'],meta['authors'],meta['doi'],
          meta['citations'],meta['cited_by'])

with db.conn.cursor() as cursor:
    cursor.execute(sql_command, args)

In [22]:
sql_command = "SELECT * FROM metadata"
# Load the data
data = pd.read_sql(sql_command, db.conn)
data

,id,url,journal,year,volume,issue,title,authors,doi,citations,cited_by
0,1,[article.url],self.journal,2001,5,8,Hello world,"[Smith, J, Woodruff, W, Harris, M]",asdfljasd;flkjasf,"[5, 9, 20, 3, 4, 40, 41, 45]","[1, 2, 6, 7, 8]"
1,7,[article.url],self.journal,2008,5,8,Hello world,"[Smith, J, Woodruff, W, Harris, M]",asdfljasd;flkjasf,"[5, 9, 20, 3, 4]","[1, 2, 6, 7, 8]"
2,8,[article.url],self.journal,2008,5,8,Hello world,"[Smith, J, Woodruff, W, Harris, M]",asdfljasd;flkjasf,"[5, 9, 20, 3, 4]","[1, 2, 6, 7, 8]"


### Find out if article is already in db

In [11]:
title = 'Hello world'
year = 2001
sql_command = """\
SELECT COUNT(*) \
FROM metadata \
WHERE title = %s \
AND year = %s"""

args = (title, year)
print(sql_command)
print(args)

SELECT COUNT(*) FROM metadata WHERE title = %s AND year = %s
('Hello world', 2001)


In [12]:
with db.conn.cursor() as cursor:
    cursor.execute(sql_command, args)
    print(cursor.fetchone()[0] > 0)

True


### Find out what the max id currently is

In [13]:
sql_command = """\
SELECT MAX(id) \
FROM metadata"""

print(sql_command)

SELECT MAX(id) FROM metadata


In [14]:
with db.conn.cursor() as cursor:
    cursor.execute(sql_command)
    nextID = (cursor.fetchone()[0])+1
print(nextID)

8


### Get index of a specific article

In [42]:
title = 'Hello world'
year = 2008
sql_command = """\
SELECT * \
FROM metadata \
WHERE title = %s \
AND year = %s"""

args = (title, year)
print(sql_command)
print(args)

with db.conn.cursor() as cursor:
    cursor.execute(sql_command, args)
    art_id = cursor.fetchone()

print(art_id)

SELECT * FROM metadata WHERE title = %s AND year = %s
('Hello world', 2008)
(7, ['article.url'], 'self.journal', 2008, 5, 8, 'Hello world', ['Smith, J', 'Woodruff, W', 'Harris, M'], 'asdfljasd;flkjasf', [5, 9, 20, 3, 4], [1, 2, 6, 7, 8])


In [33]:
art_id = []
art_id = [i[0] for i in art_id] # convert from tuple

In [48]:
art_id = list(art_id)

In [44]:
meta_keys = ['id','url','journal','year','volume','issue','title','authors','doi','citations','cited_by'] 

In [49]:
res = dict(zip(meta_keys, art_id)) 

In [50]:
res

{'authors': ['Smith, J', 'Woodruff, W', 'Harris, M'],
 'citations': [5, 9, 20, 3, 4],
 'cited_by': [1, 2, 6, 7, 8],
 'doi': 'asdfljasd;flkjasf',
 'id': 7,
 'issue': 8,
 'journal': 'self.journal',
 'title': 'Hello world',
 'url': ['article.url'],
 'volume': 5,
 'year': 2008}

In [51]:
section={}
section['intro']='lorem ipsum'
section['abstract']='hello world'
section['methods']='me may ma mo moo'

In [55]:
for i in section.keys():
    print(section[i])

lorem ipsum
hello world
me may ma mo moo


In [59]:
sql_command = "SELECT * FROM body"
# Load the data
data = pd.read_sql(sql_command, db.conn)
data

,id,sectiona,sectionb,prose
